In [0]:
print("welcome to compute")

welcome to compute


In [0]:
spark.conf.set(
  "
)

In [0]:
df_store = spark.read.parquet("abfss://containers@datab.dfs.core.windows.net/rawdata/dbo.stores.parquet")
df_store.show(5)

+--------+--------------------+------------+
|store_id|          store_name|    location|
+--------+--------------------+------------+
|       1|     City Mall Store|         UAE|
|       2|   High Street Store|Saudi Arabia|
|       3|   Tech World Outlet|       Qatar|
|       4|Cairo Festival Ci...|       Egypt|
|       5|          Mega Plaza|      Kuwait|
+--------+--------------------+------------+



In [0]:
df_product = spark.read.parquet("abfss://containers@datab.dfs.core.windows.net/rawdata/dbo.products.parquet")
df_store.show(5)

+--------+--------------------+------------+
|store_id|          store_name|    location|
+--------+--------------------+------------+
|       1|     City Mall Store|         UAE|
|       2|   High Street Store|Saudi Arabia|
|       3|   Tech World Outlet|       Qatar|
|       4|Cairo Festival Ci...|       Egypt|
|       5|          Mega Plaza|      Kuwait|
+--------+--------------------+------------+



In [0]:
df_transaction = spark.read.parquet("abfss://containers@datab.dfs.core.windows.net/rawdata/dbo.transactions.parquet")
df_store.show(5)

+--------+--------------------+------------+
|store_id|          store_name|    location|
+--------+--------------------+------------+
|       1|     City Mall Store|         UAE|
|       2|   High Street Store|Saudi Arabia|
|       3|   Tech World Outlet|       Qatar|
|       4|Cairo Festival Ci...|       Egypt|
|       5|          Mega Plaza|      Kuwait|
+--------+--------------------+------------+



In [0]:
df_customers = spark.read.parquet("abfss://containers@datab.dfs.core.windows.net/rawdata/MohammedHameds/Retails_Project/refs/heads/main/dataset/customers.parquet")
df_customers.show(5)

+-----------+----------------+--------------------+------------+-----------------+
|customer_id|       full_name|               email|     country|registration_date|
+-----------+----------------+--------------------+------------+-----------------+
|        101|    Ahmed Khaled|ahmed.khaled1@gma...|       Egypt|       2025-10-16|
|        102| Sara Al Mansour|sara.almansour2@o...|Saudi Arabia|       2025-10-18|
|        103|    Layla Kazemi|layla.kazemi3@yah...|         UAE|       2025-10-19|
|        104|     Omar Farouk|omar.farouk4@gmai...|       Egypt|       2025-10-17|
|        105|Fatima Al Rashid|fatima.alrashid5@...|Saudi Arabia|       2025-10-15|
+-----------+----------------+--------------------+------------+-----------------+
only showing top 5 rows


In [0]:
df_store.write.mode("overwrite").saveAsTable("sretails.saleess.store_bronze")
df_product.write.mode("overwrite").saveAsTable("sretails.saleess.product_bronze")
df_transaction.write.mode("overwrite").saveAsTable("sretails.saleess.transaction_bronze")
df_customers.write.mode("overwrite").saveAsTable("sretails.saleess.customer_bronze")

In [0]:
df_product.write.mode("overwrite").saveAsTable("sretails.saleess.product_bronze")

In [0]:
from pyspark.sql.functions import col,sum,count

In [0]:
df_customers=df_customers.select(
    col("customer_id").cast("int"),
    col("full_name"),
    col("email"),
    col("country"),
    col("registration_date").cast("date")
)
df_customers.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- full_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- country: string (nullable = true)
 |-- registration_date: date (nullable = true)



In [0]:
df_transaction.show(5)

+--------------+-----------+----------+--------+--------+----------------+
|transaction_id|customer_id|product_id|store_id|quantity|transaction_date|
+--------------+-----------+----------+--------+--------+----------------+
|            31|        101|         1|       1|       3|      2025-04-01|
|            32|        102|         2|       2|       2|      2025-04-03|
|            33|        103|         3|       3|       1|      2025-04-05|
|            34|        104|         4|       4|       5|      2025-04-07|
|            35|        105|         5|       5|       2|      2025-04-09|
+--------------+-----------+----------+--------+--------+----------------+
only showing top 5 rows


In [0]:
df_customer=df_customers

In [0]:
df_retails_silver = df_transaction.join(df_customer,"customer_id").join(df_product,"product_id").join(df_store, "store_id").withColumn("total_amount", col("quantity") * col("price"))
display(df_retails_silver)

In [0]:
df_retails_silver.write.mode("overwrite").saveAsTable("sretails.saleess.df_retails_silver")

In [0]:
df_country_customer = df_retails_silver.groupBy("country").agg(count("customer_id").alias("Count_Of_Customrs"))
display(df_country_customer)

country,Count_Of_Customrs
Saudi Arabia,17
UAE,16
Egypt,17


Databricks visualization. Run in Databricks to view.

In [0]:
df_country_customer.write.mode("overwrite").saveAsTable("sretails.saleess.country_customer_gold")

In [0]:
df_Contry_transaction = df_retails_silver.groupBy("country", "category").agg(count("transaction_id").alias("Count_transaction"))
display(df_Contry_transaction)

country,category,Count_transaction
Saudi Arabia,Fitness,3
UAE,Stationery,2
UAE,Electronics,5
Egypt,Electronics,7
UAE,Fitness,6
Egypt,Accessories,3
Saudi Arabia,Accessories,4
Egypt,Fitness,6
Egypt,Stationery,1
Saudi Arabia,Electronics,8


Databricks visualization. Run in Databricks to view.

In [0]:
df_Contry_transaction.write.mode("overwrite").saveAsTable("sretails.saleess.Contry_transaction_gold")

In [0]:


df_sales_per_country = df_retails_silver.groupBy("country") \
    .agg(sum("total_amount").alias("Total_Sales")) \
    .orderBy("Total_Sales", ascending=False)

display(df_sales_per_country)


country,Total_Sales
Saudi Arabia,74010
Egypt,56414
UAE,47161


In [0]:
df_sales_per_country.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("sretails.saleess.Contry_transaction_gold")

In [0]:
df_sales_per_product = df_retails_silver.groupBy("product_name") \
    .agg(sum("total_amount").alias("Total_Sales")) \
    .orderBy("Total_Sales", ascending=False)

display(df_sales_per_product)


product_name,Total_Sales
Smartwatch,69986
Dumbbell Set,33983
Laptop Stand,21978
Bluetooth Speaker,18000
Wireless Mouse,9600
Desk Organizer,6783
Pen Drive 32GB,6589
Yoga Mat,4990
Water Bottle,4186
Notebook Set,1490


In [0]:
df_sales_per_product.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("sretails.saleess.Sales_transaction_gold")

In [0]:
df_sales_per_store = df_retails_silver.groupBy("store_name") \
    .agg(sum("total_amount").alias("Total_Sales")) \
    .orderBy("Total_Sales", ascending=False)

display(df_sales_per_store)


store_name,Total_Sales
High Street Store,87986
Cairo Festival City Mall,55961
City Mall Store,13786
Tech World Outlet,11773
Mega Plaza,8079


In [0]:
df_chart = df_retails_silver.groupBy("product_name", "category") \
    .agg(sum("total_amount").alias("Total_Sales"))

display(df_chart)


product_name,category,Total_Sales
Desk Organizer,Accessories,6783
Notebook Set,Stationery,1490
Pen Drive 32GB,Electronics,6589
Smartwatch,Electronics,69986
Yoga Mat,Fitness,4990
Laptop Stand,Accessories,21978
Water Bottle,Fitness,4186
Wireless Mouse,Electronics,9600
Bluetooth Speaker,Electronics,18000
Dumbbell Set,Fitness,33983


In [0]:
df_chart.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("sretails.saleess.Chart_transaction_gold")